<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
import requests
import json
import numpy as np
from datetime import date
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries
import plotly.graph_objects as go
import plotly.io as pio
pio.templates
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
nyse = pd.read_csv('NYSE.tsv', delimiter='\t')
nasdaq = pd.read_csv('NASDAQ.tsv', delimiter='\t')

In [3]:
tickers = nyse.append(nasdaq).drop_duplicates()

In [4]:
symbols = list(tickers['Symbol'])

In [5]:
api = os.getenv("API")
ts = TimeSeries(key=api)

In [6]:
stat_options = ['1. open', '2. high', '3. low', '4. close', '5. volume']

In [7]:
def get_stock(stock, stat):
    path= f'stock/{stock}.csv'
    if not os.path.isfile(path):
        print(f'Getting API stock data for {stock}')
        data, meta_data = ts.get_weekly(stock)
        timeseries = {}
        for date, vals in data.items():
            timeseries[date]=vals
        df = pd.DataFrame.from_dict(timeseries).T
        df.to_csv(path)
    else:
        print(f'Loading cached stock data for {stock}')
        df = pd.read_csv(path, index_col=0)
    return df

In [8]:
def get_ngram(stock):
    path= f'ngram/{stock}.csv'
    if not os.path.isfile(path):
        print(f'Getting API Ngram data for {stock}')
        query = f'${stock}'
        r = requests.get(f"https://storywrangling.org/api/{query}?language=en&metric=freq").json()
        df = pd.DataFrame(r['ngramdata'][r['ngrams'][0]]['data'], columns=['date','freq']).set_index(['date'])
        df.index = pd.to_datetime(df.index,format='%Y-%m-%d')
        # df['freq']=[np.log(f) for f in df['freq']] # Convert to log-freq
        df.to_csv(path)
    else:
        print(f'Loading cached ngram data for {stock}')
        df = pd.read_csv(path, index_col=0)
    return df

In [9]:
def name_lookup(stock):
    return tickers[tickers.Symbol==stock].Description.values[0]

In [10]:
def draw_timeseries(stock, stat):
    fig = make_subplots(rows=3, cols=1, specs=[[{"secondary_y": True}],[{}],[{}]], subplot_titles=(f'Stock {stat[3:].title()} vs. Ngram Freq', f'{stock} Stock {stat[3:].title()}', f'Ngram Freq of "${stock}"'))
    try:
        stock_data = get_stock(stock, stat)
        fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data[stat], name=(f'{stock}: {stat[3:].title()}'), opacity=.5, line=dict(color='green', width=2)), secondary_y=False, row=1, col=1)
        fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data[stat], name=(f'{stock}: {stat[3:].title()}'), line=dict(color='green', width=2)), row=3, col=1)
    except:
        print(f'Could not find stock data for {stock}')
    try:
        ngram_data = get_ngram(stock)
        fig.add_trace(go.Scatter(x=ngram_data.index, y=ngram_data.freq, name=(f'${stock}: Ngram Freq'), opacity=.5, line=dict(color='orange', width=2)), secondary_y=True, row=1, col=1)
        fig.add_trace(go.Scatter(x=ngram_data.index, y=ngram_data.freq, name=(f'${stock}: Ngram Freq'), line=dict(color='orange', width=2)), row=2, col=1)
    except:
        print(f'Could not find ngram data for ${stock}')
    fig.update_layout(title_text=f'Stock & Ngram Data for {name_lookup(stock)} ({stock})', template='plotly_white',showlegend=False,height=1000)
    fig.update_xaxes(range=[np.datetime64(date(2010,1,1)), np.datetime64(date(2020,8,1))])
    return fig

In [11]:
#draw_timeseries('GOOG', '5. volume')

In [12]:
stock_options = []
for name, symbol in dict(zip(tickers.Description, tickers.Symbol)).items():
    stock_options.append({'label':str(f'{name} ({symbol})'), 'value':str(symbol)})

In [ ]:
app = dash.Dash()
app.layout = html.Div([
    dcc.Dropdown(
        id='stock-dropdown',
        options=stock_options,
        value='AAPL'
    ),
    dcc.Dropdown(
        id='stat-dropdown',
        options=[
            {'label': 'Open', 'value': '1. open'},
            {'label': 'High', 'value': '2. high'},
            {'label': 'Low', 'value': '3. low'},
            {'label': 'Close', 'value': '4. close'},
            {'label': 'Volume', 'value': '5. volume'}
        ],
        value='5. volume'
    ),
    dcc.Graph(id='timeseries-graph', figure=draw_timeseries('AAPL', '5. volume'))
])

@app.callback(
    Output('timeseries-graph', 'figure'),
    [Input('stock-dropdown', 'value'), Input('stat-dropdown', 'value')])
def update_figure(stock, stat):
    return draw_timeseries(stock, stat)

app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

Loading cached stock data for AAPL
Loading cached ngram data for AAPL
Running on http://127.0.0.1:8050/
Debugger PIN: 940-164-840
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Loading cached stock data for AAPL
Loading cached ngram data for AAPL
Loading cached stock data for AAPL
Loading cached ngram data for AAPL
Loading cached stock data for AAPL
Loading cached ngram data for AAPL
Loading cached stock data for AAPL
Loading cached ngram data for AAPL
